In [1]:
from dataset import steepest_gradient
import numpy as np


def template_matching(dataset):
    """Fully looped original implementation."""
    dataset = steepest_gradient(dataset, 1)

    def norm_corr(a1, a2):
        numerator = np.sum(a1 * a2)
        denominator = np.sqrt(
            np.sum(np.square(a1)) * np.sum(np.square(a2))
        )
        return np.divide(numerator, denominator)

    res = []
    for i in range(len(dataset.data)):
        res.append([])
        detected_peak = 0
        for j in range(len(dataset.data[i])-100):
            data = dataset.data[i][j:j+100]
            peak = dataset.data[i][dataset.peaks[i][0]-50:dataset.peaks[i][0]+50]
            out = norm_corr(np.subtract(data, np.mean(data)), np.subtract(peak, np.mean(peak)))
            if out >= 0.92 and not detected_peak-50 <= j <= detected_peak+50:
                detected_peak = j
                res[i].append(j+50)
    dataset.peaks = np.array(res)
    return dataset


def template_matching2(dataset):
    """Single loop implementation with fancy indexing.)"""
    dataset = steepest_gradient(dataset, 1)
    indexer = np.arange(100)[None, :] + np.arange(4900)[:, None]

    for i in range(len(dataset.data)):
        template = dataset.data[i][dataset.peaks[i][0]-50:dataset.peaks[i][0]+50]
        template = template - template.mean()

        data = dataset.data[i][indexer]
        data = data - np.mean(data, axis=1)[:, None]

        numerator = np.sum(data * template, axis=1)

        denominator = np.sqrt(
            np.sum(np.square(data), axis=1) * np.sum(np.square(template))
        )
        np.divide(numerator, denominator)
    return dataset


def template_matching3(dataset):
    """No loop implementation with fancy indexing.)"""
    dataset = steepest_gradient(dataset, 1)

    indexer = np.arange(100)[None, :] + np.arange(4900)[:, None]

    template_indexer = (dataset.peaks[:, 0].reshape(len(dataset.peaks), -1) - 50)+np.arange(100) 
    templates = dataset.data[:, template_indexer[-1, :]]
    templates = templates - np.mean(templates, axis=1).reshape(-1, 1)

    data = dataset.data[:, indexer]
    data = data - np.mean(data, axis=2).reshape(len(data), -1, 1)

    numerator = np.sum(np.multiply(data, templates[:, None]), axis=2)

    denominator = np.sqrt(
        np.sum(np.square(data), axis=2) * np.sum(np.square(templates), axis=1).reshape(-1, 1)
    )

    np.divide(numerator, denominator)
    return dataset

In [3]:
import timeit
from dataset import sample, ECGDataset

file = "FILE WITH ECG DATA (NUMPY ARRAY)"
samp = sample(ECGDataset(file))

# original implementation (fully looped)
%timeit template_matching(samp)

# single loop implementation with fancy indexing
%timeit template_matching2(samp)

# no loop implementation with fancy indexing
%timeit template_matching3(samp)



1 loop, best of 3: 1.94 s per loop


10 loops, best of 3: 33.2 ms per loop


10 loops, best of 3: 73.7 ms per loop
